In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pwd

/home/jovyan/ds_nudge_up/playground


In [3]:
import sys
sys.path += ["../"]

import os
import src.utils as utl
rasterio_config = utl.load_yaml_config("rasterioc_config.yaml")
os.environ.update(rasterio_config)

from src.featurize import *
from src.featurize.nn_forward_pass import featurize
import torch
import numpy as np
import pandas as pd
from pathlib import Path

## Load files

In [4]:
featurization_params = utl.load_yaml_config("featurisation.yaml")
satellite_config = utl.load_yaml_config("satellite_config.yaml")
satellite_config = satellite_config[featurization_params['satellite_search_params']['satellite_name']]
data_sources = utl.load_yaml_config('data_catalog.yaml')

In [5]:
points_gdf = utl.load_points_gdf(**data_sources['request_points'])

In [6]:
points_gdf = points_gdf[:10]

In [7]:
points_gdf

,Lat,Lon,geometry
0,34.654878,74.334486,POINT (74.33449 34.65488)
1,34.554878,74.434486,POINT (74.43449 34.55488)
2,34.454878,74.434486,POINT (74.43449 34.45488)
3,34.454878,74.384486,POINT (74.38449 34.45488)
4,34.504878,74.384486,POINT (74.38449 34.50488)
5,34.554878,74.384486,POINT (74.38449 34.55488)
6,34.554878,74.284486,POINT (74.28449 34.55488)
7,34.604878,74.234486,POINT (74.23449 34.60488)
8,34.554878,74.234486,POINT (74.23449 34.55488)
9,34.554878,74.184486,POINT (74.18449 34.55488)


## Get STAC items

In [8]:
# %%time
points_gdf_with_stac = dask_fetch_stac_items(
    points_gdf, 
    featurization_params['dask']['n_partitions'], 
    featurization_params['satellite_search_params']
)
points_gdf_with_stac = points_gdf_with_stac.sort_index().reset_index()

In [9]:
points_gdf_with_stac.stac_item.iloc[0]

ID: LC08_L2SP_150036_20140917_02_T1
"Bounding Box: [71.9823585339872, 33.51435484217948, 74.56820565024216, 35.686025157820524]"
Datetime: 2014-09-17 05:42:06.746875+00:00
datetime: 2014-09-17T05:42:06.746875Z
platform: landsat-8
"proj:bbox: [226785.0, 3712185.0, 459915.0, 3949215.0]"
proj:epsg: 32643
description: Landsat Collection 2 Level-2 Surface Reflectance Product
"instruments: ['oli', 'tirs']"
eo:cloud_cover: 3.62
view:off_nadir: 0


In [10]:
points_gdf_with_stac

,point_id,Lat,Lon,geometry,hilbert_distance,stac_item,cloud_cover,season
0,0,34.654878,74.334486,POINT (74.33449 34.65488),2526451350,<Item id=LC08_L2SP_150036_20140917_02_T1>,3.62,autumn
1,0,34.654878,74.334486,POINT (74.33449 34.65488),2526451350,<Item id=LC08_L2SP_150036_20140410_02_T1>,1.19,summer
2,0,34.654878,74.334486,POINT (74.33449 34.65488),2526451350,None,NaN,spring
3,0,34.654878,74.334486,POINT (74.33449 34.65488),2526451350,<Item id=LC08_L2SP_149036_20131212_02_T1>,7.57,winter
4,1,34.554878,74.434486,POINT (74.43449 34.55488),3221225472,<Item id=LC08_L2SP_149036_20140926_02_T1>,5.34,autumn
5,1,34.554878,74.434486,POINT (74.43449 34.55488),3221225472,<Item id=LC08_L2SP_149036_20140606_02_T1>,5.21,summer
6,1,34.554878,74.434486,POINT (74.43449 34.55488),3221225472,None,NaN,spring
7,1,34.554878,74.434486,POINT (74.43449 34.55488),3221225472,<Item id=LC08_L2SP_149036_20131212_02_T1>,7.57,winter
8,2,34.454878,74.434486,POINT (74.43449 34.45488),4294967295,None,NaN,spring
9,2,34.454878,74.434486,POINT (74.43449 34.45488),4294967295,<Item id=LC08_L2SP_149036_20131212_02_T1>,7.57,winter


## Create features

In [24]:
data_loader = create_data_loader(points_gdf_with_stac, satellite_config, featurization_params['batch_size'])

In [25]:
device = torch.device(featurization_params['device'])

In [26]:
model = RCF(
    featurization_params['num_features'], 
    featurization_params['kernel_size'], 
    len(satellite_config['bands'])
).eval().to(device)

In [27]:
X_features = create_features(
    data_loader, 
    featurization_params['num_features'], 
    len(points_gdf_with_stac), 
    model, 
    device, 
    satellite_config['min_image_edge']
)

  0%|          | 0/8 [00:00<?, ?it/s]

TypeError: expected Tensor as element 2 in argument 0, but got NoneType

## Save to file

In [16]:
ROOT = Path.cwd().parent
output_folder_path = ROOT/"data/01_preprocessed/mosaiks_features/"
os.makedirs(output_folder_path, exist_ok=True)

In [17]:
X_features_df = pd.DataFrame(X_features, columns=[str(c) for c in range(X_features.shape[1])])
gdf = pd.concat([points_gdf_with_stac.drop("stac_item", axis=1), X_features_df], axis=1)

In [18]:
filename = "mosaiks_points_latest"
gdf.to_parquet(output_folder_path / f"{filename}.parquet.gzip", compression="gzip")